# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

## Data

That's where we load and preprocess our data.

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

Buffer_size = 1

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(Buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 10000

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(BATCH_SIZE)
test_data = test_data.batch(BATCH_SIZE)

validation_inputs, validation_targets = next(iter(validation_data))

### Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 300

model = tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape = (28,28,1)),
                           tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                           tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),  
                           tf.keras.layers.Dense(output_size, activation = 'softmax')
                           ])

### Choose the optimizer and loss function

In [4]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training

In [5]:
NUM_EPOCHS = 7

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data, 
          epochs = NUM_EPOCHS,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          validation_steps=1, verbose=2)

Epoch 1/7
6/6 - 7s - loss: 1.8172 - accuracy: 0.5529 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/7
6/6 - 4s - loss: 0.8794 - accuracy: 0.8172 - val_loss: 0.5288 - val_accuracy: 0.8492
Epoch 3/7
6/6 - 4s - loss: 0.4606 - accuracy: 0.8694 - val_loss: 0.3747 - val_accuracy: 0.8912
Epoch 4/7
6/6 - 4s - loss: 0.3569 - accuracy: 0.8971 - val_loss: 0.3178 - val_accuracy: 0.9053
Epoch 5/7
6/6 - 4s - loss: 0.3071 - accuracy: 0.9131 - val_loss: 0.2790 - val_accuracy: 0.9165
Epoch 6/7
6/6 - 4s - loss: 0.2703 - accuracy: 0.9233 - val_loss: 0.2527 - val_accuracy: 0.9277
Epoch 7/7
6/6 - 4s - loss: 0.2423 - accuracy: 0.9309 - val_loss: 0.2324 - val_accuracy: 0.9320


In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.2266 - accuracy: 0.9338


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.23. Test accuracy: 93.38%
